# N-Queens 
### The N-queens problem is to place n queens on a N x N chessboard, such that no queen attacks another.

In [1]:
def moveGen(n, row, col):
  """
  Generate all 8 connected moves for a position in nxn chess board.
  
  Parametes:
  ----------
  n: size of the chess board (int).
  row: index of row (int).
  col: index of clumn (int).
  """
  
  moves = []
  if col+1 < n: moves.append((row, col+1))
  if row+1 < n and col+1 < n: moves.append((row+1, col+1))
  if row+1 < n: moves.append((row+1, col))
  if row+1 < n and col-1 >= 0: moves.append((row+1, col-1))
  if col-1 >= 0: moves.append((row, col-1))
  if row-1 >= 0 and col-1 >= 0: moves.append((row-1, col-1))
  if row-1 >= 0: moves.append((row-1, col))
  if row-1 >= 0 and col+1 < n: moves.append((row-1, col+1))
  return moves
  
def genDigs(n, row, col):
  """
  Generate all digonal positions for a given position in an nxn board
  
  Parameters:
  -----------
  n: size of the chess board (int).
  row: index of row (int).
  col: index of clumn (int).
  """
  
  down_row = row + 1
  right_col = col + 1
  up_row = row - 1
  left_col = col - 1
  digs = []
  
  while up_row >= 0 or left_col >= 0 or down_row < n  or right_col < n:
    if down_row < n and right_col < n:
      digs.append((down_row, right_col))
    if down_row < n and left_col >= 0:
      digs.append((down_row, left_col))
    if up_row >= 0 and right_col < n:
      digs.append((up_row, right_col))
    if up_row >= 0 and left_col >= 0:
      digs.append((up_row, left_col))

    down_row += 1
    right_col += 1
    up_row -= 1
    left_col -= 1
  return digs
    

def isPositionSafe(qpos, n, row, col):
  """
  Returns true if a given position is safe to place a queen.
  
  Parameters:
  -----------
  n: size of the chess board (int).
  row: index of row (int).
  col: index of clumn (int).
  qpos: list of positions that are already occupied.
  """
  digonals = genDigs(n, row, col)
  for pos in qpos:
    if row == pos[0]:return False
    if col == pos[1]: return False
    if pos in digonals: return False
  return True

# nQueen state space search

This method uses BFS as a search technique to find a solution.

In [134]:
def nQueen(queens):
  visit = [(0,0)]
  visit_count = 0
  count = 0
  iteration = 0
  closed = []
  queenPositions = []
  
  def search():
    nonlocal visit_count, visit
    while len(visit) > 0:
      current = visit.pop(0)
      visit_count += 1
      closed.append(current)
      if isPositionSafe(queenPositions, queens, *current):
        queenPositions.append(current)
      next_moves = moveGen(queens, *current)
      next_moves = set(next_moves) - set(closed)
      next_moves = set(next_moves) - set(visit)
      visit += next_moves
  
  while len(queenPositions) < queens:
    iteration += 1
    if len(queenPositions) == 0:
      if count > (queens * queens) // 2:
        break
      else:
        count += 1
    search()
#     if iteration % 5 == 0:
#       print("iteration:", iteration)
#       print(queenPositions, len(queenPositions), visit_count)

    if len(queenPositions) < queens:
      last_position = queenPositions[-1]
      queenPositions.pop()
      idx = closed.index(last_position)
      del closed[idx+1:]
      visit = moveGen(queens, *last_position)
      visit = list(set(visit) - set(closed))

  print("states visited: ", visit_count)
  print("queens placed: ", len(queenPositions))
  return queenPositions

# pos = nQueen(10)
# print("No Solution" if len(pos) == 0 else pos)

# Heuristic best first serarch nQueen

This is an implementation of best first search on top of BFS.

In [2]:
import math

def nQueenHuristic(queens):
  visit = [(0,0)]
  visit_count = 0
  count = 0
  iteration = 0
  closed = []
  queenPositions = []
  
  def search():
    nonlocal visit_count, visit
    
    while len(visit) != 0:
      current = visit.pop(0)
      visit_count += 1
      closed.append(current)

      if isPositionSafe(queenPositions, queens, *current):
        queenPositions.append(current)

      next_moves = moveGen(queens, *current)
      next_moves = set(next_moves) - set(closed)
      next_moves = list(set(next_moves) - set(visit))
      
      if len(next_moves) > 0:
        filtered_moves = list(filter(lambda node: isPositionSafe(queenPositions, queens, *node), next_moves))
        if len(filtered_moves) == 0: 
          filtered_moves = list(sorted(next_moves, key = lambda node: math.sqrt((current[0] - node[0])**2 + (current[1] - node[1])**2)))
        visit = filtered_moves
 
  while len(queenPositions) < queens:
    iteration += 1
    if len(queenPositions) == 0:
      if count > (queens * queens) // 2:
        break
      else:
        count += 1
    search()
    
    if len(queenPositions) < queens:
      last_position = queenPositions[-1]
      queenPositions.pop()
      idx = closed.index(last_position)
      del closed[idx+1:]
      visit = moveGen(queens, *last_position)
      visit = list(set(visit) - set(closed))

  print("states visited: ", visit_count)
  print("queens placed: ", len(queenPositions))
  return queenPositions

# Hill climbing perturbation search

In [30]:
from random import randint

def randomPosition(n):
  return (randint(0, n-1), randint(0, n-1))
  
def randomlyPlaceQueen(n):
  q_locs = []
  for i in range(n):
    q_locs.append(randomPosition(n))
  return q_locs

def huristic(state, n):
  h = 0
  for pos in state:
    if isPositionSafe(state, n, *pos): h += 1
  return h

def findSafePos(state, n):
  open = [(0,0)]
  closed = set()
  
  while len(open) > 0:
    current = open.pop(0)
    closed.add(current)
    if isPositionSafe(state, n, *current):
      return current
    
    next_moves = moveGen(n, *current)
    next_moves = set(next_moves) - set(closed)
    next_moves = set(next_moves) - set(open)
    open += next_moves
  
  return None

def findUnsafeQueen(current_state, n):
  idx = 0
  safe = True
  while safe and idx < len(current_state):
    safe = isPositionSafe(current_state, n, *current_state[idx])
    if safe: idx += 1
  return idx

This is an implementation of hill climbing algorithm. The algorithm tris to find the best path to take but usually get stuck with partial solution.

In [39]:
def nQueen(n):
  current_state = randomlyPlaceQueen(n)
  print("init state: ", current_state)
  h = huristic(current_state, n)
  
  while huristic(current_state, n) < n:
    safe_pos = findSafePos(current_state, n)
    print("safe pos: ", safe_pos)
    if safe_pos is not None:
      idx = findUnsafeQueen(current_state, n)
      current_state[idx] = safe_pos
    
    print(current_state)
    break
  return current_state, h

nQueen(5)

init state:  [(2, 0), (4, 4), (1, 1), (3, 4), (1, 3)]
safe pos:  None
[(2, 0), (4, 4), (1, 1), (3, 4), (1, 3)]


([(2, 0), (4, 4), (1, 1), (3, 4), (1, 3)], 0)

This is an implementation of iterative hill climbing.

In [52]:
def iterativenQueen(n, epoch=10):
  state = randomlyPlaceQueen(n)
  print("state: ", state)
  best_state = state
  
  for  i in range(epoch):
    state = randomlyPlaceQueen(n)
    new_state = state.copy()
    safe_pos = findSafePos(state, n)
    if safe_pos is not None:
      idx = findUnsafeQueen(state, n)
      new_state[idx] = safe_pos
    
    while huristic(new_state, n) > huristic(state, n):
      state = new_state
      new_state = state.copy()
      safe_pos = findSafePos(state, n)
      if safe_pos is not None:
        idx = findUnsafeQueen(current_state, n)
        new_state[idx] = safe_pos
      
      if huristic(new_state, n) > huristic(best_state, n):
        best_state = new_state

  print(huristic(best_state, n))
  return best_state

In [53]:
iterativenQueen(5)

state:  [(0, 2), (1, 3), (3, 3), (4, 3), (2, 3)]
0


[(0, 2), (1, 3), (3, 3), (4, 3), (2, 3)]